Reference: https://www.kaggle.com/reynaldo/naive-xgb/comments/notebook

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import model_selection, preprocessing
from xgboost.sklearn import XGBRegressor
import datetime
import math
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

pd.options.display.max_columns = 300
%matplotlib inline  

In [17]:
df_train = pd.read_csv('train.csv')#, parse_dates=['timestamp'])
df_test = pd.read_csv('test.csv')#, parse_dates=['timestamp'])
df_macro = pd.read_csv('macro.csv')#, parse_dates=['timestamp'])
id_test = df_test.id
df_train.sample(3)

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_av

In [28]:
trainsub = df_train[df_train.timestamp < '2015-01-01']
trainsub = trainsub[trainsub.product_type=="Investment"]

ind_1m = trainsub[trainsub.price_doc <= 1000000].index
ind_2m = trainsub[trainsub.price_doc == 2000000].index
ind_3m = trainsub[trainsub.price_doc == 3000000].index

train_index = set(df_train.index.copy())

for ind, gap in zip([ind_1m, ind_2m, ind_3m], [10, 3, 2]):
    ind_set = set(ind)
    ind_set_cut = ind.difference(set(ind[::gap]))

    train_index = train_index.difference(ind_set_cut)

train_cv = df_train.loc[train_index]
y_train_cv = train_cv["price_doc"]
x_train_cv = train_cv.drop(["id", "timestamp", "price_doc"], axis=1)
for c in x_train_cv.columns:
    if x_train_cv[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train_cv[c].values)) 
        x_train_cv[c] = lbl.transform(list(x_train_cv[c].values))
        #x_train.drop(c,axis=1,inplace=True)

In [18]:
y_train = df_train["price_doc"]
x_train = df_train.drop(["id", "timestamp", "price_doc"], axis=1)
x_test = df_test.drop(["id", "timestamp"], axis=1)

#can't merge train with test because the kernel run for very long time

for c in x_train.columns:
    if x_train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train[c].values)) 
        x_train[c] = lbl.transform(list(x_train[c].values))
        #x_train.drop(c,axis=1,inplace=True)
        
for c in x_test.columns:
    if x_test[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_test[c].values)) 
        x_test[c] = lbl.transform(list(x_test[c].values))
        #x_test.drop(c,axis=1,inplace=True)   

In [41]:
# xreg = XGBRegressor(learning_rate=0.05,
#                     max_depth = 5,
#                     subsample = 0.7,
#                     colsample_bytree = 0.7,
#                     objective = 'reg:linear',
#                     nthread = 3,
#                     silent= 1)  # and for classifier  
# xreg.fit(x_train, y_train)  
import scipy.stats as st

one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

params = {  
    "n_estimators": st.randint(3, 40),
    "max_depth": st.randint(3, 40),
    "learning_rate": st.uniform(0.05, 0.4),
    "colsample_bytree": one_to_left,
    "subsample": one_to_left,
    "gamma": st.uniform(0, 10),
    'reg_alpha': from_zero_positive,
    "min_child_weight": from_zero_positive,
}
iters = 100

xgbreg = XGBRegressor()

In [42]:
def rmsle_XGB(labels, preds):
    labels = labels.tolist()
    preds = preds.tolist()
#     print len(labels), len(preds)
    terms_to_sum = [(math.log(labels[i] + 1) - math.log(max(0, preds[i]) + 1)) ** 2.0 for i, pred in enumerate(labels)]
    return (sum(terms_to_sum) * (1.0 / len(preds))) ** 0.5

rmsle_scorer = make_scorer(rmsle_XGB, greater_is_better=False)#, needs_proba=True)

In [43]:
gs = RandomizedSearchCV(xgbreg, params, n_iter=iters, n_jobs=10, scoring='neg_mean_squared_error')#rmsle_scorer)
gs.fit(x_train, y_train)

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
          fit_params={}, iid=True, n_iter=100, n_jobs=10,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7f25a241d0>, 'subsample': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7f259f9b10>, 'reg_alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7f259f9190>, 'colsample_bytree': <...at 0x7f7f259f9910>, 'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7f25a17490>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring='neg_mean_squared_erro

In [44]:
gs.cv_results_

{'mean_fit_time': array([  29.02505565,   40.075032  ,   37.9133211 ,   15.51472799,
          20.24163946,   43.36784832,   38.24391643,   11.05326088,
          27.02457833,   17.83668876,   10.78660695,    9.20893645,
           7.06088257,   40.79732211,   19.89215732,   22.37896029,
          31.50768741,    9.55200569,   33.07290435,   37.75113002,
          25.4235634 ,   28.49153964,   24.29527275,    9.03075504,
          58.0494411 ,  134.27281666,   18.09241684,   17.95495303,
          77.283535  ,   22.06253393,   55.94525297,    6.53936593,
          44.72466469,   55.73750162,   46.05700906,   20.23389729,
          55.96061198,   47.14765922,   12.04121574,   62.84991797,
          12.2809844 ,   59.32038164,   48.20728572,   30.90270162,
           6.61033901,   25.77599629,   25.01727541,   20.1300567 ,
           7.45427871,   10.83392866,   46.21015557,   56.98665134,
           5.74288861,    7.64211432,   15.0833834 ,   16.88210169,
          35.58467126,   40.476

In [45]:
gs.best_score_

-8519186547577.3965

In [46]:
gs.best_estimator_
# XGBRegressor(base_score=0.5, colsample_bylevel=1,
#        colsample_bytree=0.90376743790350766, gamma=0.82229270633305096,
#        learning_rate=0.1059352041807113, max_delta_step=0, max_depth=39,
#        min_child_weight=34.807721810404857, missing=None, n_estimators=30,
#        nthread=-1, objective='reg:linear', reg_alpha=25.265495286611461,
#        reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
#        subsample=0.85828449473027224)

XGBRegressor(base_score=0.5, colsample_bylevel=1,
       colsample_bytree=0.90071921829374502, gamma=9.3323935183905053,
       learning_rate=0.1245328155683267, max_delta_step=0, max_depth=8,
       min_child_weight=17.164892931627829, missing=None, n_estimators=32,
       nthread=-1, objective='reg:linear', reg_alpha=47.982587859201288,
       reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=0.93292160321651763)

In [47]:
cv_scores = cross_val_score(gs.best_estimator_, x_train_cv, y_train_cv, scoring=rmsle_scorer, cv=5)
print cv_scores
print np.mean(cv_scores)

[-0.38293661 -0.29189308 -0.31714514 -0.29953456 -0.35775042]
-0.329851962979


In [55]:
cv = False
if cv:
    model = gs.best_estimator_.fit(x_train_cv, y_train_cv)
    preds = model.predict(x_train_cv)
    print rmsle_XGB(preds, y_train_cv)
else:
    model = gs.best_estimator_.fit(x_train, y_train)
    preds = model.predict(x_train)
    print rmsle_XGB(preds, y_train)

0.437146103755


In [7]:
# xgb_params = {
#     'eta': 0.05,
#     'max_depth': 5,
#     'subsample': 0.7,
#     'colsample_bytree': 0.7,
#     'objective': 'reg:linear',
#     'eval_metric': 'rmse',
#     'silent': 1
# }

# dtrain = xgboost.DMatrix(x_train, y_train)
# dtest = xgb.DMatrix(x_test)

In [56]:
y_predict = model.predict(x_test)
output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})
output.head()

,id,price_doc
0,30474,5494059.0
1,30475,8616312.0
2,30476,5692932.0
3,30477,6024692.5
4,30478,5078892.0


In [57]:
output.shape

(7662, 2)

In [58]:
output.to_csv('submissions/xgb_sklearn_subset100_mse_alldata.csv', index=False)